In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
from torch.utils.data import DataLoader,random_split
from torchvision.datasets import MNIST
from torchvision import transforms as tfr
import matplotlib.pyplot as plt
import pandas as pd


In [3]:
ts = tfr.Compose([
    tfr.ToTensor(),
    tfr.Normalize(0.5,0.5),
    tfr.Resize((64,64))
    
])
root ='data'
ds = MNIST(root, train= True, download=True,transform=ts)
test_df = MNIST(root, train=False, download=True, transform=ts)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
ds[0][0].shape

torch.Size([1, 64, 64])

In [5]:
tr_length = int(len(ds) * 0.9)
tr_ds, val_ds = random_split(ds, [tr_length, len(ds) - tr_length])
print(f"Train datasetda {len(tr_ds)} rasm bor.")
print(f"Validation datasetda {len(val_ds)} rasm bor.")
print(f"Test datasetda {len(test_df)} rasm bor.")

Train datasetda 54000 rasm bor.
Validation datasetda 6000 rasm bor.
Test datasetda 10000 rasm bor.


In [6]:
bs = 128
tr_dl = DataLoader(dataset = tr_ds, batch_size = bs, shuffle = True, drop_last = False, num_workers = 4)
val_dl = DataLoader(dataset = val_ds, batch_size = bs, shuffle = False, drop_last = False, num_workers = 4)
test_dl = DataLoader(dataset = test_df, batch_size = bs, shuffle = False, drop_last = False, num_workers = 4)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
print(f"Train dataloaderda {len(tr_dl)} ta batch bor.") 
print(f"Validation dataloaderda {len(val_dl)} ta batch bor.")
print(f"Test dataloaderda {len(test_dl)} ta batch bor.") 

Train dataloaderda 422 ta batch bor.
Validation dataloaderda 47 ta batch bor.
Test dataloaderda 79 ta batch bor.


In [8]:
im_size = 64
in_chs = 1
out_chs = 8
ks = 3
pad = 1 
pool_size = 2
num_classes = 10

in_fs = (im_size // 32) * (im_size // 32)* out_chs * 8
print(in_fs)


256


In [9]:
class Model(torch.nn.Module):
    
    def __init__(self, in_chs, out_chs, in_fs, ks, pad, ps, n_cls): 
        super(Model, self).__init__()
        
        self.in_chs, self.out_chs, self.ks, self.pad, self.ps, self.n_cls = in_chs, out_chs, pad, ks, ps, n_cls
        
        self.conv_1 = torch.nn.Conv2d(in_channels =in_chs, out_channels =out_chs, kernel_size = ks, padding = pad)
        self.act_1 = torch.nn.ReLU()
        self.pool_1 = torch.nn.MaxPool2d(kernel_size = ps, stride = ps)
        
        self.conv_2 = torch.nn.Conv2d(in_channels =out_chs, out_channels =out_chs*2, kernel_size = ks, padding = pad)
        self.act_2 = torch.nn.ReLU()
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=ps, stride=ps)
        
        self.conv_3 = torch.nn.Conv2d(in_channels=out_chs*2, out_channels=out_chs*4, kernel_size=ks, padding=pad)
        self.act_3 = torch.nn.ReLU()
        self.pool_3 = torch.nn.MaxPool2d(kernel_size=ps, stride=ps)
        
        self.conv_4 = torch.nn.Conv2d(in_channels=out_chs*4, out_channels=out_chs*6, kernel_size=ks, padding=pad)
        self.act_4 = torch.nn.ReLU()
        self.pool_4 = torch.nn.MaxPool2d(kernel_size=ps, stride=ps) 
        
        
        self.conv_5 = torch.nn.Conv2d(in_channels=out_chs*6, out_channels=out_chs*8, kernel_size=ks, padding=pad)
        self.act_5 = torch.nn.ReLU()
        self.pool_5 = torch.nn.MaxPool2d(kernel_size=ps, stride=ps)
        
        
        
        self.fc = torch.nn.Linear(in_features = in_fs, out_features = in_fs//2)
        self.fc1 = torch.nn.Linear(in_features=in_fs//2, out_features=in_fs//4)
        self.out = torch.nn.Linear(in_features = in_fs//4, out_features = n_cls)
        self.softmax = torch.nn.Softmax(dim = 1)
        
    def forward(self, inp):
        
        conv_1 = self.conv_1(inp) 
        act_1 = self.act_1(conv_1)
        out_1 = self.pool_1(act_1)
        
        conv_2 = self.conv_2(out_1)
        act_2 = self.act_2(conv_2)
        out_2 = self.pool_2(act_2)
        
        conv_3 = self.conv_3(out_2)
        act_3 = self.act_3(conv_3)
        out_3 = self.pool_3(act_3)
        
        conv_4 = self.conv_4(out_3)
        act_4  = self.act_4(conv_4)
        out_4  = self.pool_4(act_4)
        
        conv_5 = self.conv_5(out_4)
        act_5  = self.act_5(conv_5)
        out_5  = self.pool_5(act_5)
          
        
        bs = inp.shape[0]
        flat = out_5.view(bs, -1)
        fc_1 = self.fc(flat)
        fc_2 = self.fc1(fc_1)
        out =self.out(fc_2)
        
        #fc_1 = self.act(self.fc(flat))
        #fc_2 = self.act_2(self.fc1(fc_1))
        #out = self.softmax(self.out(fc_2))
        
        return self.softmax(out)
    
m = Model(in_chs = in_chs, out_chs = out_chs, in_fs = in_fs, ks = ks, pad = pad, ps = pool_size, n_cls = num_classes)
m

Model(
  (conv_1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_1): ReLU()
  (pool_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_3): ReLU()
  (pool_3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_4): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_4): ReLU()
  (pool_4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_5): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act_5): ReLU()
  (pool_5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=256, out_features=128, bias=True)
  (fc1): Linear(in_f

In [10]:



    
m = Model(in_chs = in_chs, out_chs = out_chs, in_fs = in_fs, ks = ks, pad = pad, ps = pool_size, n_cls = num_classes)
x = torch.rand(1, in_chs, im_size, im_size)
m(x)



tensor([[0.1020, 0.1101, 0.1023, 0.0976, 0.1054, 0.0885, 0.1032, 0.0969, 0.0996,
         0.0943]], grad_fn=<SoftmaxBackward0>)

In [11]:
def train_setup(m): return m.to("cuda"), 10, "cuda", torch.nn.CrossEntropyLoss(), torch.optim.Adam(params = m.parameters(), lr = 0.001)

In [12]:
m, epochs, device, loss_fn, optimizer = train_setup(m)

In [13]:
from tqdm import tqdm

print("Train boshlandi...")
best_acc = 0
for epoch in range(epochs):
    
    epoch_loss, epoch_acc, total = 0, 0, 0
    for idx, batch in tqdm(enumerate(tr_dl)):
        ims, gts = batch
        ims, gts = ims.to(device), gts.to(device)
        
        # print(ims.shape)
        # print(gts.shape)
        total += ims.shape[0]
        
        preds = m(ims)
        _, pred_cls = torch.max(preds.data, dim = 1)
        # print(pred_cls)
        # print(gts)
        loss = loss_fn(preds, gts)
        # loss = loss_fn(preds, gts.unsqueeze(1))
        
        epoch_acc += (pred_cls == gts).sum().item()
        epoch_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    tr_loss = epoch_loss / len(tr_dl)
    print(f"Epoch {epoch + 1} train jarayoni tugadi")
    print(f"Epoch {epoch + 1} dagi train loss -> {(tr_loss):.3f}")
    print(f"Epoch {epoch + 1} dagi train accuracy -> {(epoch_acc / total):.3f}")
    
    m.eval()
    with torch.no_grad():
        val_epoch_loss, val_epoch_acc, val_total = 0, 0, 0
        for idx, batch in enumerate(val_dl):
            ims, gts = batch
            ims, gts = ims.to(device), gts.to(device)
            val_total += ims.shape[0]

            preds = m(ims)
            loss = loss_fn(preds, gts)
            _, pred_cls = torch.max(preds.data, dim = 1)
            val_epoch_acc += (pred_cls == gts).sum().item()
            val_epoch_loss += loss.item()
        
        val_acc = val_epoch_acc / val_total
        val_loss = val_epoch_loss / len(val_dl)
        
        print(f"Epoch {epoch + 1} validation jarayoni tugadi")
        print(f"Epoch {epoch + 1} dagi validation loss -> {(val_loss):.3f}")
        print(f"Epoch {epoch + 1} dagi validation accuracy -> {val_acc:.3f}")
        
        
        if val_acc > best_acc:
            os.makedirs("saved_models", exist_ok=True)
            best_acc = val_acc
            torch.save(m.state_dict(), f"saved_models/cifar_best_model_{val_acc:.3f}.pth")        

Train boshlandi...


422it [00:21, 19.88it/s]

Epoch 1 train jarayoni tugadi
Epoch 1 dagi train loss -> 1.644
Epoch 1 dagi train accuracy -> 0.820


Epoch 1 validation jarayoni tugadi
Epoch 1 dagi validation loss -> 1.508
Epoch 1 dagi validation accuracy -> 0.954


422it [00:16, 24.84it/s]

...

422it [00:17, 23.60it/s]

Epoch 9 train jarayoni tugadi
Epoch 9 dagi train loss -> 1.480
Epoch 9 dagi train accuracy -> 0.981


Epoch 9 validation jarayoni tugadi
Epoch 9 dagi validation loss -> 1.476
Epoch 9 dagi validation accuracy -> 0.986


422it [00:16, 24.89it/s]


Epoch 10 train jarayoni tugadi
Epoch 10 dagi train loss -> 1.482
Epoch 10 dagi train accuracy -> 0.980
Epoch 10 validation jarayoni tugadi
Epoch 10 dagi validation loss -> 1.483
Epoch 10 dagi validation accuracy -> 0.978
